In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

# Data For PINNs

In [2]:
N_train = 5000
N_bound = 100

np.random.seed(123)
train_x_y_t = lhs(3,N_train)
np.random.seed(12345)
bound_x_y = lhs(2,N_bound)
bound_x_y_t = np.concatenate((bound_x_y,np.zeros((N_bound,1))),1)

In [3]:
train_x_y_t = torch.from_numpy(train_x_y_t).float()
bound_x_y = torch.from_numpy(bound_x_y).float()
bound_x_y_t = torch.from_numpy(bound_x_y_t).float()

x = train_x_y_t[:,0:1]
y = train_x_y_t[:,1:2]
t = train_x_y_t[:,2:3]
x.requires_grad_()
y.requires_grad_()
t.requires_grad_()

tensor([[0.3976],
        [0.7691],
        [0.9756],
        ...,
        [0.2023],
        [0.6423],
        [0.6873]], requires_grad=True)

## Get Data

In [4]:
def relative_l2(u_pred,u_real):
    l2 = np.linalg.norm(u_real-u_pred,2)/np.linalg.norm(u_real,2)
    return l2

In [5]:
import scipy.integrate
from numpy import exp
from math import sqrt
import math
def InIn(x_y_t):
    x1 = x_y_t[:,0:1]
    x2 = x_y_t[:,1:2]
    t = x_y_t[:,2:3]
    P = np.zeros_like(t)
    for i in range(len(t)):
        f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t[i]))*(y1-y2)
        p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
        P[i] = p/(4*np.pi*t[i])
    return P

In [6]:
np.random.seed(5678)
test_x_y_t = lhs(3,500)
x1 = test_x_y_t[:,0:1]
x2 = test_x_y_t[:,1:2]
t_ = test_x_y_t[:,2:3]
test_u = np.zeros_like(t_)
for i in range(len(t_)):
    f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t_[i]))*(y1-y2)
    p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
    test_u[i] = p/(4*np.pi*t_[i])


In [7]:
test_x_y_t = torch.from_numpy(test_x_y_t).float()

### Labeled Data

In [8]:
np.random.seed(234)
n = 6
gan_data_x_t = lhs(3,10)
np.random.seed(1234)
chosen_indices = np.random.choice(gan_data_x_t.shape[0], size=n, replace=False)
gan_data_x_t = gan_data_x_t[chosen_indices, :]
gan_data_u = InIn(gan_data_x_t)

In [9]:
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
gan_data_u = torch.from_numpy(gan_data_u).float()

# Method2
**$\text { GA - PINN }$**

<div class="alert alert-info">`loss_function：`

$\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\mathrm{L}_{\text {PINN }}:=\mathrm{L}_f+\lambda_1 \mathrm{~L}_b \text { with } \mathrm{L}_b:=\sum_{i=1}^I \mathrm{~L}_{b_i}
$



</div>

In [10]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs2 = NN_H2(3, 100, 4, 1)
PINNs2.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs2.parameters()}], lr=1e-3)

discriminator= get_discriminator(4, 100, 1, 1)
discriminator.apply(weights_init)
optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.01}], lr=5e-3)

loss_all_2 = []
test_loss_2 = []

#########gpu############
discriminator.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs2.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
#########gpu############

loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs2.cuda()
    
    ##############loss D############
    pre_H = PINNs2(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))
    
    loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
    
    optimizer2.zero_grad()
    loss_d .backward()
    optimizer2.step()  
    
    ##############loss G ############
    pre_H = PINNs2(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    loss_L = torch.mean(torch.square(pre_H - gan_data_u))+torch.mean(1-d_fake)
    
    optimizer1.zero_grad()
    loss_L.backward()
    optimizer1.step()  
    
    ##### loss_Bi  ######
    u_bound = PINNs2(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    
    u_inside = PINNs2(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    

    loss = loss_bound+loss_f
    #loss = loss_f 
    loss1_value = loss.item()
    
    loss_all_2.append(loss1_value)
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs2.cpu()
    test_loss =  relative_l2(PINNs2(test_x_y_t).detach().numpy(),test_u)
    test_loss_2.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item())
    it = it + 1        
loss1_value    

It: 0 Loss: 0.15083111822605133
It: 100 Loss: 0.0007569971494376659
It: 200 Loss: 0.00011366924445610493
It: 300 Loss: 0.00015833126963116229
It: 400 Loss: 5.048021921538748e-05
It: 500 Loss: 0.0002168232313124463
It: 600 Loss: 5.153549136593938e-05
It: 700 Loss: 2.7267542463960126e-05
It: 800 Loss: 5.165429683984257e-05
It: 900 Loss: 2.594945726741571e-05
It: 1000 Loss: 6.293693149928004e-05
It: 1100 Loss: 2.4106269847834483e-05
It: 1200 Loss: 6.696966738672927e-05
It: 1300 Loss: 2.321913962077815e-05
It: 1400 Loss: 5.954224616289139e-05
It: 1500 Loss: 1.9700917619047686e-05
It: 1600 Loss: 9.787188173504546e-05
It: 1700 Loss: 2.1630852643284015e-05
It: 1800 Loss: 6.484812183771282e-05
It: 1900 Loss: 1.0329483302484732e-05
It: 2000 Loss: 8.390376024181023e-06


4.9240279622608796e-06

# Method3
$\text { PINN }^{\dagger}$

<div class="alert alert-info">objective function：

$\overline{\mathrm{L}}_{\text {PINN }} =\mathrm{L}_{\text {PINN }}+\lambda_2\mathrm{L}_T$



</div>

In [11]:
k = 2

In [12]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs3 = NN_H2(3, 100, 4, 1)
PINNs3.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs3.parameters()}], lr=1e-3)

loss_all_3 = []
test_loss_3 = []

#########gpu############
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs3.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
#########gpu############

loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs3.cuda()
    
    ##### loss_Bi  ######
    u_bound = PINNs3(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    u_inside = PINNs3(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))

    #####loss PI#######
    loss = loss_bound+loss_f
    pre_H = PINNs3(gan_data_x_t)
    
    loss_p = k*torch.mean(torch.square(pre_H - gan_data_u))+loss
    
    loss1_value = loss.item()
    
    loss_all_3.append(loss1_value)
    optimizer1.zero_grad()
    loss_p.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs3.cpu()
    test_loss =  relative_l2(PINNs3(test_x_y_t).detach().numpy(),test_u)
    test_loss_3.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item(), 'testloss:', test_loss)
    it = it + 1        
loss1_value    

It: 0 Loss: 0.32263725996017456 testloss: 0.87101925302898
It: 100 Loss: 0.0015345608117058873 testloss: 0.10525667053468309
It: 200 Loss: 0.0006534883286803961 testloss: 0.0842062301911943
It: 300 Loss: 0.000306295114569366 testloss: 0.07748983904418336
It: 400 Loss: 0.00013968560961075127 testloss: 0.061109594487760696
It: 500 Loss: 7.779165753163397e-05 testloss: 0.06264984170348234
It: 600 Loss: 5.682200571754947e-05 testloss: 0.06416518511107946
It: 700 Loss: 0.0009080457966774702 testloss: 0.09049847162558756
It: 800 Loss: 8.114730007946491e-05 testloss: 0.0532151282117644
It: 900 Loss: 5.3114370530238375e-05 testloss: 0.056398095694341475
It: 1000 Loss: 4.095479744137265e-05 testloss: 0.05797647543801355
It: 1100 Loss: 3.4287601010873914e-05 testloss: 0.05860628737489541
It: 1200 Loss: 0.0001038218688336201 testloss: 0.044067666739054066
It: 1300 Loss: 5.197549762669951e-05 testloss: 0.048305893044671165
It: 1400 Loss: 3.648856727522798e-05 testloss: 0.05060400666575947
It: 1500

4.996566985937534e-06

# Method4 
$\text { GA - PINN }^{\dagger}$

<div class="alert alert-info">objective function：

$
\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\overline{\mathrm{L}}_{\text {PINN }} =\mathrm{L}_{\text {PINN }}+\lambda_2\mathrm{L}_T
$



</div>


In [13]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs4 = NN_H2(3, 100, 4, 1)
PINNs4.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs4.parameters()}], lr=1e-3)

discriminator= get_discriminator(4, 100, 1, 1)
discriminator.apply(weights_init)
optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.01}], lr=5e-3)


loss_all_4 = []
test_loss_4 = []

#########gpu############
discriminator.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs4.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
#########gpu############


loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs4.cuda()
    
    ##############loss D############
    pre_H = PINNs4(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))
    
    loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
    
    optimizer2.zero_grad()
    loss_d .backward()
    optimizer2.step()  
    
    ##############loss G############
    pre_H = PINNs4(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    loss_L = torch.mean(torch.square(pre_H - gan_data_u))+torch.mean(1-d_fake)
    
    optimizer1.zero_grad()
    loss_L.backward()
    optimizer1.step()  

    ##### loss_Bi  ######
    u_bound = PINNs4(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    
    u_inside = PINNs4(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    
    #####loss PI#######
    loss = loss_bound+loss_f
    pre_H = PINNs4(gan_data_x_t)
    
    loss_p = k*torch.mean(torch.square(pre_H - gan_data_u))+loss
    
    loss1_value = loss.item()
    
    loss_all_4.append(loss1_value)
    optimizer1.zero_grad()
    loss_p.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs4.cpu()
    test_loss =  relative_l2(PINNs4(test_x_y_t).detach().numpy(),test_u)
    test_loss_4.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item(), 'testloss:', test_loss)
    it = it + 1        
loss1_value    

It: 0 Loss: 0.15083111822605133 testloss: 0.9964910628676131
It: 100 Loss: 0.00096617586677894 testloss: 0.05917623496735388
It: 200 Loss: 0.0008984272717498243 testloss: 0.03280606293337447
It: 300 Loss: 0.00033834358328022063 testloss: 0.02866524151276716
It: 400 Loss: 0.00015967989747878164 testloss: 0.024999520631530013
It: 500 Loss: 0.00029455695766955614 testloss: 0.021419667841735664
It: 600 Loss: 0.0001466119138058275 testloss: 0.019839651891530707
It: 700 Loss: 7.493895100196823e-05 testloss: 0.018871504486494262
It: 800 Loss: 0.00016039240290410817 testloss: 0.01614244979127432
It: 900 Loss: 7.123992691049352e-05 testloss: 0.015943465451868415
It: 1000 Loss: 7.890559936640784e-05 testloss: 0.013976063271017811
It: 1100 Loss: 9.124869393417612e-06 testloss: 0.01383212192742338
It: 1200 Loss: 1.9706771126948297e-05 testloss: 0.013964679771277293
It: 1300 Loss: 9.587460226612166e-06 testloss: 0.012874051988149433
It: 1400 Loss: 6.140713594504632e-06 testloss: 0.01198047346153154

4.998561053071171e-06

In [14]:
test_loss_2 = np.array(test_loss_2)
test_loss_3 = np.array(test_loss_3)
test_loss_4 = np.array(test_loss_4)

In [16]:
np.save('../../experimental_data/J_6/test_loss_2',test_loss_2)
np.save('../../experimental_data/J_6/test_loss_3',test_loss_3)
np.save('../../experimental_data/J_6/test_loss_4',test_loss_4)